# 02. Policy Comparison Analysis

Compare KPI metrics across different execution policies:
- Execution path: NPU-only vs NPU+FB vs GPU-only
- Frequency: 1Hz vs 5Hz vs 10Hz
- Warm-up: enabled vs disabled

In [ ]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from parse_logs import load_log, calculate_metrics, find_steady_state
from plot_kpi import setup_style, compare_experiments

setup_style()
%matplotlib inline

## 1. Load All Experiment Data

In [ ]:
DATA_DIR = Path('../data')

# Define experiments (update with your actual files)
experiments = {
    'NPU-only 10Hz': 'npu_10hz_nw_*.csv',
    'NPU+FB 10Hz': 'fb_10hz_nw_*.csv',
    'GPU-only 10Hz': 'gpu_10hz_nw_*.csv',
    'NPU+FB 5Hz': 'fb_5hz_nw_*.csv',
    'NPU+FB 1Hz': 'fb_1hz_nw_*.csv',
    'NPU+FB Warm-up': 'fb_10hz_w_*.csv',
}

# Load data
data = {}
for name, pattern in experiments.items():
    files = list(DATA_DIR.glob(pattern))
    if files:
        data[name] = load_log(str(files[0]))
        print(f"Loaded {name}: {len(data[name])} records")
    else:
        print(f"No files found for {name}")

print(f"\nLoaded {len(data)} experiments")

## 2. Calculate Metrics for Each Experiment

In [ ]:
results = []

for name, df in data.items():
    # Use steady-state data (after 30s)
    steady_df = find_steady_state(df, warm_up_seconds=30.0)
    metrics = calculate_metrics(steady_df)
    
    results.append({
        'experiment': name,
        'latency_p50': metrics.latency_p50,
        'latency_p95': metrics.latency_p95,
        'latency_std': metrics.latency_std,
        'thermal_slope': metrics.thermal_slope,
        'thermal_max': metrics.thermal_max,
        'power_mean': metrics.power_mean,
        'memory_peak': metrics.memory_peak,
        'inference_count': metrics.inference_count
    })

results_df = pd.DataFrame(results)
results_df.set_index('experiment', inplace=True)
results_df

## 3. Execution Path Comparison

In [ ]:
# Filter path comparison experiments
path_experiments = ['NPU-only 10Hz', 'NPU+FB 10Hz', 'GPU-only 10Hz']
path_df = results_df.loc[results_df.index.isin(path_experiments)]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Latency
ax = axes[0]
x = np.arange(len(path_df))
width = 0.35
ax.bar(x - width/2, path_df['latency_p50'], width, label='P50', color='steelblue')
ax.bar(x + width/2, path_df['latency_p95'], width, label='P95', color='coral')
ax.set_ylabel('Latency (ms)')
ax.set_title('Latency by Execution Path')
ax.set_xticks(x)
ax.set_xticklabels(path_df.index, rotation=45, ha='right')
ax.legend()

# Thermal
ax = axes[1]
colors = ['green' if s < 0.5 else 'orange' if s < 1.0 else 'red' 
          for s in path_df['thermal_slope']]
ax.bar(x, path_df['thermal_slope'], color=colors)
ax.set_ylabel('Thermal Slope (°C/min)')
ax.set_title('Thermal Increase Rate')
ax.set_xticks(x)
ax.set_xticklabels(path_df.index, rotation=45, ha='right')

# Power
ax = axes[2]
ax.bar(x, path_df['power_mean'], color='green')
ax.set_ylabel('Average Power (mW)')
ax.set_title('Power Consumption')
ax.set_xticks(x)
ax.set_xticklabels(path_df.index, rotation=45, ha='right')

plt.tight_layout()
plt.show()

## 4. Frequency Impact Analysis

In [ ]:
# Filter frequency experiments
freq_experiments = ['NPU+FB 1Hz', 'NPU+FB 5Hz', 'NPU+FB 10Hz']
freq_df = results_df.loc[results_df.index.isin(freq_experiments)]

if len(freq_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    x = np.arange(len(freq_df))
    
    # Thermal slope vs frequency
    ax = axes[0]
    ax.bar(x, freq_df['thermal_slope'], color='orange')
    ax.set_ylabel('Thermal Slope (°C/min)')
    ax.set_title('Thermal vs Frequency')
    ax.set_xticks(x)
    ax.set_xticklabels(freq_df.index)
    
    # Power vs frequency
    ax = axes[1]
    ax.bar(x, freq_df['power_mean'], color='green')
    ax.set_ylabel('Average Power (mW)')
    ax.set_title('Power vs Frequency')
    ax.set_xticks(x)
    ax.set_xticklabels(freq_df.index)
    
    plt.tight_layout()
    plt.show()

## 5. Warm-up Effect

In [ ]:
# Compare warm-up vs no warm-up
warmup_experiments = ['NPU+FB 10Hz', 'NPU+FB Warm-up']
warmup_df = results_df.loc[results_df.index.isin(warmup_experiments)]

if len(warmup_df) >= 2:
    print("Warm-up Effect:")
    print(warmup_df[['latency_p50', 'latency_p95', 'latency_std']].T)
    
    # Check if latency variance decreased with warm-up
    no_warmup_std = warmup_df.loc['NPU+FB 10Hz', 'latency_std']
    warmup_std = warmup_df.loc['NPU+FB Warm-up', 'latency_std']
    
    improvement = (no_warmup_std - warmup_std) / no_warmup_std * 100
    print(f"\nLatency std improvement with warm-up: {improvement:.1f}%")

## 6. Summary Table

In [ ]:
# Format summary table
summary = results_df.copy()
summary['latency_p50'] = summary['latency_p50'].map('{:.1f}'.format)
summary['latency_p95'] = summary['latency_p95'].map('{:.1f}'.format)
summary['thermal_slope'] = summary['thermal_slope'].map('{:.2f}'.format)
summary['power_mean'] = summary['power_mean'].map('{:.0f}'.format)

summary[['latency_p50', 'latency_p95', 'thermal_slope', 'power_mean', 'memory_peak']]

## 7. Key Findings

In [ ]:
print("=" * 50)
print("KEY FINDINGS")
print("=" * 50)

if len(results_df) > 0:
    # Best latency
    best_latency = results_df['latency_p50'].idxmin()
    print(f"\n1. Best Latency (P50): {best_latency}")
    print(f"   Value: {results_df.loc[best_latency, 'latency_p50']:.1f} ms")
    
    # Best thermal
    best_thermal = results_df['thermal_slope'].idxmin()
    print(f"\n2. Best Thermal: {best_thermal}")
    print(f"   Slope: {results_df.loc[best_thermal, 'thermal_slope']:.2f} °C/min")
    
    # Best power
    best_power = results_df['power_mean'].idxmin()
    print(f"\n3. Best Power: {best_power}")
    print(f"   Average: {results_df.loc[best_power, 'power_mean']:.0f} mW")